# 영화 추천 시스템

In [2]:
import pandas as pd
import numpy as np

df = pd.read_json('all_movies.json')

In [3]:
df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count
0,False,/2Nti3gYAX513wvhp8IiLL6ZDyOm.jpg,"[10751, 35, 12, 14]",950387,en,A Minecraft Movie,왕년의 게임 챔피언이었지만 지금은 폐업 직전의 게임샵 주인이 된 '개릿'과 엄마를 ...,868.2685,/k5aQ2TqKcQFwPoXHkpAGoKNVDLZ.jpg,2025-03-31,A MINECRAFT MOVIE 마인크래프트 무비,False,6.516,1182
1,False,/tyfO9jHgkhypUFizRVYD0bytPjP.jpg,"[10751, 14]",447273,en,Snow White,눈보라가 몰아치던 겨울 밤 태어난 백설공주. 온정이 넘치던 왕국에서 모두의 사랑을 ...,387.4897,/7EruTgYzDFM8kWhr2ysuuwoXFqk.jpg,2025-03-12,백설공주,False,4.300,808
2,False,/j0NUh5irX7q2jIRtbLo8TZyRn6y.jpg,"[27, 9648, 35]",574475,en,Final Destination Bloodlines,끝없는 악몽에 시달리던 ‘스테파니’는 그 원인을 찾던 중 집안 대대로 내려오는 죽...,386.4101,/oS0b0iiwEW2T9vcGHFYpTlTPmuI.jpg,2025-05-09,파이널 데스티네이션: 블러드라인,False,7.380,171
3,False,/cJvUJEEQ86LSjl4gFLkYpdCJC96.jpg,"[10752, 28]",1241436,en,Warfare,"""실화 기반, 전장의 숨결을 생생히 담아낸 리얼 타임 전쟁 드라마"" 2006년 이...",379.8887,/srj9rYrjefyWqkLc6l2xjTGeBGO.jpg,2025-04-09,워페어,False,7.119,324
4,False,/fTrQsdMS2MUw00RnzH0r3JWHhts.jpg,"[28, 80, 53]",1197306,en,A Working Man,전직 블랙 옵스 요원이었던 레본 케이드(제이슨 스타뎀)는 평범한 건설 노동자로 살아...,357.3065,/tK5sCN4cebvTt9MuSlNHehVQnYg.jpg,2025-03-26,워킹맨,False,6.500,748


In [4]:
df.shape

(9979, 14)

In [5]:
df.columns

Index(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'title', 'video', 'vote_average', 'vote_count'],
      dtype='object')

In [6]:
c = df['vote_average'].mean()
c

6.263448742358954

In [7]:
m = df['vote_count'].quantile(0.9)
m

4680.000000000004

In [8]:
q_movies = df.copy().loc[df['vote_count'] >= m]
q_movies.shape

(998, 14)

In [9]:
q_movies['vote_count'].sort_values()

243      4684
1887     4685
1599     4685
2397     4691
789      4693
        ...  
124     31789
174     32229
130     33836
86      37112
173     37452
Name: vote_count, Length: 998, dtype: int64

In [10]:
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    return (v / (v + m) * R) + (m / (m + v) * c)

In [11]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [12]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies[['title', 'genre_ids', 'id', 'vote_count', 'vote_average', 'score']].head(10)

,title,genre_ids,id,vote_count,vote_average,score
135,쇼생크 탈출,"[18, 80]",278,28279,8.710,8.362603
49,대부,"[18, 80]",238,21437,8.686,8.251894
130,다크 나이트,"[18, 28, 80, 53]",155,33836,8.519,8.244933
86,인터스텔라,"[12, 18, 878]",157336,37112,8.455,8.209583
329,펄프 픽션,"[53, 80]",680,28655,8.500,8.186004
208,포레스트 검프,"[35, 18, 10749]",13,28195,8.468,8.154166
209,파이트 클럽,[18],550,30248,8.438,8.146632
173,인셉션,"[28, 878, 12]",27205,37452,8.369,8.135116
268,반지의 제왕: 왕의 귀환,"[12, 14, 28]",122,25064,8.484,8.134613
201,반지의 제왕: 반지 원정대,"[12, 14, 28]",120,25957,8.421,8.091420


In [13]:
q_movies[['title', 'genre_ids', 'id','poster_path', 'vote_count', 'vote_average', 'score']].to_json("movies_top10.json", orient='records', force_ascii=False, indent=2)